In [ ]:
# %%python -m venv llama3_env
# %llama3_env/Scripts/activate  # On Windows, use `llama3_env\Scripts\activate`
%pip install ollama torch transformers


In [ ]:
import ollama

# Load the Llama3:70b model

#system prompt
messages = [{
    "role": "user",
    "content": "why is the sky blue?"
}]
model_name = "llama3:70b"
response = ollama.chat(model=model_name, messages=messages)

print(response['messages']['content'])

In [ ]:
# use text classification to distinguish between positive integer overflow and negative integer overflow (none)
from torch.utils.data import DataLoader, Dataset
from transformers import DataCollatorForLanguageModeling, LlamaTokenizer

# Load the tokenizer for Llama3
tokenizer = LlamaTokenizer.from_pretrained(model_name)

class TextDataset(Dataset):
    def __init__(self, tokenizer, texts, max_length):
        self.input_ids = [tokenizer.encode(text, truncation=True, max_length=max_length, return_tensors="pt")[0] for text in texts]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return self.input_ids[idx]

# Example texts for fine-tuning
texts = [
    "def reserve(buffer, new_size):\n    if new_size > len(buffer):\n        buffer.extend([None] * (new_size - len(buffer)))\n    return buffer",
    "def reserve(buffer, new_size):\n    old_size = len(buffer)\n    if new_size > old_size:\n        buffer.extend([None] * (new_size - old_size))\n    return buffer"
]

# Create dataset
dataset = TextDataset(tokenizer, texts, max_length=512)

# Create data loader
data_loader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=DataCollatorForLanguageModeling(tokenizer))
